In [3]:
# !pip install langchain
# !pip install langchain-community
# !pip install pymupdf
# !pip install faiss-cpu
# !pip install transformers
# !pip install sentence-transformers

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from langchain.retrievers.multi_query import MultiQueryRetriever
import torch  # 추가: GPU 메모리 관리 및 디바이스 설정

# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("SPRi AI Brief 5월호 산업동향.pdf")
docs = loader.load()

# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

# 단계 3: 임베딩(Embedding) 생성
embeddings = HuggingFaceEmbeddings(
    model_name="BM-K/KoSimCSE-roberta-multitask",
    model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"}  # 추가: 디바이스 자동 설정
)

# 단계 4: DB 생성(Create DB) 및 저장
vectorstore = Chroma.from_documents(
    documents=split_documents,
    embedding=embeddings,
    persist_directory="./chroma_db",
    collection_metadata={"hnsw:construction_ef": 100},
)

# 단계 5: Multi-Query Retriever 설정
# 변경: MultiQueryRetriever를 사용하여 다중 질문을 기반으로 문서 검색
multi_query_prompt = PromptTemplate(
    input_variables=["question"],
    template="""당신은 질문 생성자입니다.
    사용자의 질문과 관련된 다양한 관점의 질문을 한국어로 5개만 생성하세요.
    질문은 원래 질문과 의미가 유사해야 하며, 문서에서 관련 정보를 더 잘 검색할 수 있도록 재구성해야 합니다.
    각 질문은 번호를 붙여 구분하세요:

    원래 질문: {question}"""
)

model_name = "beomi/Llama-3-Open-Ko-8B-Instruct-preview"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,  # 추가: 패딩 토큰 ID 설정
    do_sample=False,
    temperature=0.0,
    return_full_text=False
)
llm = HuggingFacePipeline(pipeline=llm_pipeline)

# 변경: MultiQueryRetriever 초기화
multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(
        search_type="similarity",  # 추가: 유사도 기반 검색 명시
        search_kwargs={"k": 4}  # 추가: 상위 4개 문서 반환
    ),
    llm=llm,
    prompt=multi_query_prompt
)

# 추가: 답변 생성을 위한 프롬프트
answer_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""당신은 주어진 컨텍스트를 기반으로 질문에 답변하는 AI 어시스턴트입니다.
    답변은 간결하고 정확해야 하며, 한국어로 작성하세요.
    컨텍스트에 없는 정보는 사용하지 마세요. 정보가 없으면 "컨텍스트에 관련 정보가 없습니다."라고 답변하세요.

    #Context:
    {context}

    #Question:
    {question}

    #Answer:"""
)


# 변경: 디버깅용 체인 수정, 5개 질문만 출력하도록 설정
def parse_questions(output):
    # 출력에서 질문만 추출 (번호가 붙은 5개 질문)
    lines = output.strip().split("\n")
    questions = [line for line in lines if line.strip().startswith(("1.", "2.", "3.", "4.", "5."))]
    return "\n".join(questions[:4])  # 최대 4개 질문 반환

custom_multiquery_chain = (
    {"question": RunnablePassthrough()} 
    | multi_query_prompt 
    | llm 
    | StrOutputParser()
    | parse_questions  # 추가: 질문 파싱 함수로 5개 질문만 출력
)

No sentence-transformers model found with name BM-K/KoSimCSE-roberta-multitask. Creating a new one with mean pooling.
Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.26it/s]
Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
/tmp/ipykernel_283/3956875414.py:62: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=llm_pipeline)


In [3]:
question = 'AI 산업동향은 어때?'
multi_queries = custom_multiquery_chain.invoke(question)
multi_queries

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'1.\n    1.1. AI 산업의 현재 상태는 어떻습니까?\n    1.2. AI 산업의 미래는 어떻습니까?\n    1.3. AI 산업의 발전 방향은 무엇입니까?'